In [298]:
import os
import re
import matplotlib.pyplot as plt
import numpy as np

from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy import select

from db_scripts.brewbrain_db import BREWBRAIN_DB_ENGINE_STR, Base, Grain, CoreGrain, RecipeML
from beer_util_functions import coarsegrindasis_to_pkl

In [ ]:
def mash_point_liters(grainATs, efficiency):
  """Calculate the total point-liters from all the given grains in a mash accounting for brewhouse efficiency.
  Args:
    grainATs (List[RecipeMLGrainAT]): A list of all Recipe-to-Grain Association Table entries for the recipe.
    efficiency (float): The brewhouse efficiency as a percentage in [0,1].
  Returns:
    float: The total point-liters contributed by the mash.
  """
  kg_total = 0.0
  pkl_total  = 0.0
  for grainAT in grainATs:
    kg_total += grainAT.amount
    coarse_grid_as_is = grainAT.calc_coarse_grid_as_is()
    pkl_total += coarsegrindasis_to_pkl(coarse_grid_as_is)
  return pkl_total * kg_total * efficiency # (points/kg/L) * Kg * efficiency = (points*L)

def mash_points(mash_point_liters, volume_l):
  """Calculate the total points of a mash at a given volume in Liters.
  Args:
    mash_point_liters (float): The total point-liters contributed by the mash.
    volume_l (float): The volume to calculate the points at (typically pre-boil volume), in Liters.
  Returns:
    float: The total points contributed by the mash for the given volume.
  """
  return mash_point_liters / volume_l

In [ ]:
'''
engine = create_engine(BREWBRAIN_DB_ENGINE_STR, echo=False, future=True)
Base.metadata.create_all(engine)

# Fast test for basic recipe calculations
with Session(engine) as session:
  recipe = session.scalars(select(RecipeML).filter_by(RecipeML.style.core_style.id == 1)).first()
  
  # Mash contributions
  mash_ptliters = mash_point_liters(recipe.grains, recipe.efficiency)
  preboil_mash_pts = mash_points(mash_ptliters, recipe.preboil_vol)
  
  # Adjunct contributions...
  # TODO: We will need to determine where specific adjuncts are added
  # Stages: Mash, Post-boil, Fermenter
  for adjunctAT in recipe.adjuncts:
    if adjunctAT.stage
'''
  